# 🔹 UFC Fight Predictor Model Training

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 1. Import Libraries and Setup Environment

In [1]:
# Import necessary libraries
import os
import sys
import pandas as pd

pd.set_option('display.max_colwidth', 200) 

# Get the current working directory
current_dir = os.getcwd()

# Navigate to the project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Import from /src
sys.path.append(os.path.join(project_root, 'src'))
from utils.metrics import *
from models.model_factory import *
from utils.io_model import *
from utils.helpers import *
from models.config import *

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 2. Load Data

In [2]:
# Load UFCData
try:
    UFCData = load_data()
except Exception as e:
    print_header(f"Error loading training data: {e}", color='bright_red')

📦 UFCData object loaded from: /home/mlioi/ufc-predictor/data/processed/ufc_data.pkl


In [3]:
UFCData.summary()

📊 UFC Dataset Summary
----------------------------------------
🧪 Total samples      : 6541
🧪 Train/Test split  : 5232 / 1309
🧪 Total features     : 39

🔢 Numerical features : 34
🔠 Categorical features: 5
    - Binary          : 2
    - Multiclass      : 3

🏷 Label distribution (raw):
   - Class 0: 3795 (58.0%)
   - Class 1: 2746 (42.0%)

✅ No missing values detected

📈 Feature summary statistics (train set):
                        mean      std      min       max
NumberOfRounds         3.187    0.580    3.000     5.000
BlueAvgSigStrLanded   19.800   18.830    0.000   154.000
BlueAvgSigStrPct       0.453    0.105    0.000     1.000
BlueAvgSubAtt          0.500    0.634    0.000     8.400
BlueAvgTDLanded        1.315    1.261    0.000    10.610
BlueAvgTDPct           0.324    0.224    0.000     1.000
RedAvgSigStrLanded    21.121   19.175    0.000   141.000
RedAvgSigStrPct        0.460    0.096    0.000     1.000
RedAvgSubAtt           0.536    0.684    0.000     8.400
RedAvgTDLanded    

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 3. 🔧 Hyperparameters Tuning 🔧

In [4]:
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

hyperparameter_tuning = True

if hyperparameter_tuning:
    print_header('Hyperparameters Tuning is Activated', color = 'bright_blue')
    model_params = {
    "Support Vector Machine": (
        SVC(probability=True),
        {'C': [1, 5], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['auto']}
    ),
    "Random Forest": (
        RandomForestClassifier(),
        {'n_estimators': [10, 50, 100, 1000], 'max_depth': [3, 5, 10, 100]}
    ),
    "Logistic Regression": (
        LogisticRegression(),
        {'C': [1, 5, 10], 'solver': ['liblinear', 'lbfgs']}
    ),
    "K-Nearest Neighbors": (
        KNeighborsClassifier(),
        {'n_neighbors': [3, 6, 9, 12, 15], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}
    ),
    "AdaBoost": (
        AdaBoostClassifier(),
        {'n_estimators': [10, 50, 100], 'learning_rate': [0.01, 0.1, 1.0]}
    ),
    "Naive Bayes": (
        GaussianNB(),
        {'var_smoothing': [1e-8, 1e-7, 1e-6, 1e-5]}
    ),
    "Gradient Boosting": (
        GradientBoostingClassifier(),
        {'n_estimators': [10, 50, 100], 'learning_rate': [0.01, 0.1, 1.0], 'max_depth': [3, 5, 7]}
    ),
    "Extra Trees": (
        ExtraTreesClassifier(),
        {'n_estimators': [50, 100], 'max_depth': [None, 10, 20]}
    ),
    "Quadratic Discriminant Analysis": (
        QuadraticDiscriminantAnalysis(),
        {'reg_param': [0.1, 1]}
    ),
    "Neural Network": (
        MLPClassifier(max_iter=200, random_state=42),
        {
        # Architecture: number and size of hidden layers
        'hidden_layer_sizes': [
            (200,), (50, 50),
        ],
        # Activation function for hidden layers
        'activation': ['relu', 'logistic'],
        # Optimizer for gradient descent
        'solver': ['adam', 'sgd'],
        # L2 regularization strength (higher alpha reduces overfitting but can cause underfitting)
        'alpha': [0.001],
        # Learning rate schedule
        'learning_rate': ['adaptive'],
        # Initial learning rate (for adam and sgd solvers)
        'learning_rate_init': [0.01],
        # Early stopping based on validation performance to prevent overfitting
        'early_stopping': [True],
        # Mini-batch size for training with adam/sgd
        'batch_size': [32],
        # Momentum for SGD (controls contribution of previous updates)
        'momentum': [0.8],
        # Proportion of training set used as validation for early stopping
        'validation_fraction': [0.15]
        }
    ),
    "XGBoost": (
            XGBClassifier(eval_metric='logloss'),
            {
                'n_estimators': [50, 100],
                'learning_rate': [0.01, 0.1, 0.2],
                'max_depth': [3, 5, 7],
                'subsample': [0.8, 1.0],
                'colsample_bytree': [0.8, 1.0]
            }
        )
}
    display_model_params_table(model_params)
else:
    print_header('Hyperparameters Tuning Off', color = 'bright_red')
    model_params = None # Default Settings

╔═══════════════════════════════════════╗
║  Hyperparameters Tuning is Activated  ║
╚═══════════════════════════════════════╝


,Model,Hyperparameters
0,Support Vector Machine,"C: [1, 5]; kernel: ['linear', 'rbf', 'poly']; gamma: ['auto']"
1,Random Forest,"n_estimators: [10, 50, 100, 1000]; max_depth: [3, 5, 10, 100]"
2,Logistic Regression,"C: [1, 5, 10]; solver: ['liblinear', 'lbfgs']"
3,K-Nearest Neighbors,"n_neighbors: [3, 6, 9, 12, 15]; weights: ['uniform', 'distance']; metric: ['euclidean', 'manhattan']"
4,AdaBoost,"n_estimators: [10, 50, 100]; learning_rate: [0.01, 0.1, 1.0]"
5,Naive Bayes,"var_smoothing: [1e-08, 1e-07, 1e-06, 1e-05]"
6,Gradient Boosting,"n_estimators: [10, 50, 100]; learning_rate: [0.01, 0.1, 1.0]; max_depth: [3, 5, 7]"
7,Extra Trees,"n_estimators: [50, 100]; max_depth: [None, 10, 20]"
8,Quadratic Discriminant Analysis,"reg_param: [0.1, 1]"
9,Neural Network,"hidden_layer_sizes: [(200,), (50, 50)]; activation: ['relu', 'logistic']; solver: ['adam', 'sgd']; alpha: [0.001]; learning_rate: ['adaptive']; learning_rate_init: [0.01]; early_stopping: [True]; ..."


<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 4. 🔹 UFC Machine Learning Training

### 4.1 KNN Model Training

#### 🚀 KNN Training 

In [5]:
model_name = 'K-Nearest Neighbors'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔═══════════════════════════════════════════════════╗
║  [K-Nearest Neighbors] UFC GridSearchCV Training  ║
╚═══════════════════════════════════════════════════╝


INFO:root:[K-Nearest Neighbors] 🤖 Training...


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.550 total time=   0.1s
[CV 2/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.528 total time=   0.1s
[CV 3/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.561 total time=   0.1s
[CV 4/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.528 total time=   0.1s
[CV 5/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.545 total time=   0.1s
[CV 1/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.549 total time=   0.0s
[CV 2/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.528 total time=   0.0s
[CV 3/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.560 total time=   0.0s
[CV 4/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.528 total time=   0.0s
[CV 5/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.544 to

INFO:root:[K-Nearest Neighbors] 🔍 Best Score: 0.5793
[K-Nearest Neighbors] 🔍 Best Params: {'metric': 'euclidean', 'n_neighbors': 12, 'weights': 'uniform'}


#### 🔍 KNN Metrics

In [6]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [7]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [8]:
save_model(model, name=file_model_name[model_name])

✅ Model K-Nearest Neighbors saved to: /home/mlioi/ufc-predictor/models/knn_best.pkl


### 4.2 Support Vector Machine Model 

#### 🚀 Support Vector Machine Training 

In [9]:
model_name = 'Support Vector Machine'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔══════════════════════════════════════════════════════╗
║  [Support Vector Machine] UFC GridSearchCV Training  ║
╚══════════════════════════════════════════════════════╝


INFO:root:[Support Vector Machine] 🤖 Training...


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ....C=1, gamma=auto, kernel=linear;, score=0.603 total time=  10.6s
[CV 2/5] END ....C=1, gamma=auto, kernel=linear;, score=0.597 total time=   9.7s
[CV 3/5] END ....C=1, gamma=auto, kernel=linear;, score=0.598 total time=   9.9s
[CV 4/5] END ....C=1, gamma=auto, kernel=linear;, score=0.593 total time=  10.0s
[CV 5/5] END ....C=1, gamma=auto, kernel=linear;, score=0.593 total time=  10.0s
[CV 1/5] END .......C=1, gamma=auto, kernel=rbf;, score=0.617 total time=   5.9s
[CV 2/5] END .......C=1, gamma=auto, kernel=rbf;, score=0.576 total time=   5.9s
[CV 3/5] END .......C=1, gamma=auto, kernel=rbf;, score=0.620 total time=   6.1s
[CV 4/5] END .......C=1, gamma=auto, kernel=rbf;, score=0.588 total time=   5.9s
[CV 5/5] END .......C=1, gamma=auto, kernel=rbf;, score=0.613 total time=   5.9s
[CV 1/5] END ......C=1, gamma=auto, kernel=poly;, score=0.595 total time=   5.3s
[CV 2/5] END ......C=1, gamma=auto, kernel=poly;,

INFO:root:[Support Vector Machine] 🔍 Best Score: 0.6028
[Support Vector Machine] 🔍 Best Params: {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}


#### 🔍 Support Vector Machine Metrics

In [10]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [11]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [12]:
save_model(model, name=file_model_name[model_name])

✅ Model Support Vector Machine saved to: /home/mlioi/ufc-predictor/models/svm_best.pkl


### 4.3 Logistic Regression Model

#### 🚀 Logistic Regression Training

In [13]:
model_name = 'Logistic Regression'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔═══════════════════════════════════════════════════╗
║  [Logistic Regression] UFC GridSearchCV Training  ║
╚═══════════════════════════════════════════════════╝


INFO:root:[Logistic Regression] 🤖 Training...


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END .............C=1, solver=liblinear;, score=0.612 total time=   0.1s
[CV 2/5] END .............C=1, solver=liblinear;, score=0.599 total time=   0.1s
[CV 3/5] END .............C=1, solver=liblinear;, score=0.619 total time=   0.1s
[CV 4/5] END .............C=1, solver=liblinear;, score=0.597 total time=   0.1s
[CV 5/5] END .............C=1, solver=liblinear;, score=0.615 total time=   0.1s
[CV 1/5] END .................C=1, solver=lbfgs;, score=0.612 total time=   0.0s
[CV 2/5] END .................C=1, solver=lbfgs;, score=0.600 total time=   0.0s
[CV 3/5] END .................C=1, solver=lbfgs;, score=0.619 total time=   0.0s
[CV 4/5] END .................C=1, solver=lbfgs;, score=0.595 total time=   0.0s
[CV 5/5] END .................C=1, solver=lbfgs;, score=0.613 total time=   0.0s
[CV 1/5] END .............C=5, solver=liblinear;, score=0.611 total time=   0.1s
[CV 2/5] END .............C=5, solver=liblinear;,

/home/mlioi/anaconda3/envs/ufc_env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/5] END .................C=5, solver=lbfgs;, score=0.617 total time=   0.0s
[CV 4/5] END .................C=5, solver=lbfgs;, score=0.596 total time=   0.0s
[CV 5/5] END .................C=5, solver=lbfgs;, score=0.616 total time=   0.0s
[CV 1/5] END ............C=10, solver=liblinear;, score=0.611 total time=   0.1s
[CV 2/5] END ............C=10, solver=liblinear;, score=0.597 total time=   0.1s
[CV 3/5] END ............C=10, solver=liblinear;, score=0.617 total time=   0.1s
[CV 4/5] END ............C=10, solver=liblinear;, score=0.596 total time=   0.1s
[CV 5/5] END ............C=10, solver=liblinear;, score=0.616 total time=   0.1s


/home/mlioi/anaconda3/envs/ufc_env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mlioi/anaconda3/envs/ufc_env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

[CV 1/5] END ................C=10, solver=lbfgs;, score=0.612 total time=   0.0s
[CV 2/5] END ................C=10, solver=lbfgs;, score=0.598 total time=   0.0s
[CV 3/5] END ................C=10, solver=lbfgs;, score=0.616 total time=   0.0s
[CV 4/5] END ................C=10, solver=lbfgs;, score=0.596 total time=   0.0s
[CV 5/5] END ................C=10, solver=lbfgs;, score=0.615 total time=   0.0s


INFO:root:[Logistic Regression] 🔍 Best Score: 0.6082
[Logistic Regression] 🔍 Best Params: {'C': 1, 'solver': 'liblinear'}


#### 🔍 Logistic Regression Metrics

In [14]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [15]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [16]:
save_model(model, name=file_model_name[model_name])

✅ Model Logistic Regression saved to: /home/mlioi/ufc-predictor/models/lr_best.pkl


### 4.4 Random Forest Model

#### 🚀 Random Forest Training

In [17]:
model_name = 'Random Forest'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔═════════════════════════════════════════════╗
║  [Random Forest] UFC GridSearchCV Training  ║
╚═════════════════════════════════════════════╝


INFO:root:[Random Forest] 🤖 Training...


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ......max_depth=3, n_estimators=10;, score=0.591 total time=   0.0s
[CV 2/5] END ......max_depth=3, n_estimators=10;, score=0.594 total time=   0.0s
[CV 3/5] END ......max_depth=3, n_estimators=10;, score=0.595 total time=   0.0s
[CV 4/5] END ......max_depth=3, n_estimators=10;, score=0.577 total time=   0.0s
[CV 5/5] END ......max_depth=3, n_estimators=10;, score=0.589 total time=   0.0s
[CV 1/5] END ......max_depth=3, n_estimators=50;, score=0.591 total time=   0.2s
[CV 2/5] END ......max_depth=3, n_estimators=50;, score=0.591 total time=   0.2s
[CV 3/5] END ......max_depth=3, n_estimators=50;, score=0.588 total time=   0.2s
[CV 4/5] END ......max_depth=3, n_estimators=50;, score=0.587 total time=   0.2s
[CV 5/5] END ......max_depth=3, n_estimators=50;, score=0.593 total time=   0.2s
[CV 1/5] END .....max_depth=3, n_estimators=100;, score=0.595 total time=   0.3s
[CV 2/5] END .....max_depth=3, n_estimators=100;

INFO:root:[Random Forest] 🔍 Best Score: 0.6067
[Random Forest] 🔍 Best Params: {'max_depth': 10, 'n_estimators': 1000}


#### 🔍 Random Forest Metrics

In [18]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [19]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [20]:
save_model(model, name=file_model_name[model_name])

✅ Model Random Forest saved to: /home/mlioi/ufc-predictor/models/rf_best.pkl


### 4.5 AdaBoost Model

#### 🚀 AdaBoost Training

In [21]:
model_name = 'AdaBoost'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔════════════════════════════════════════╗
║  [AdaBoost] UFC GridSearchCV Training  ║
╚════════════════════════════════════════╝


INFO:root:[AdaBoost] 🤖 Training...


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END learning_rate=0.01, n_estimators=10;, score=0.580 total time=   0.1s
[CV 2/5] END learning_rate=0.01, n_estimators=10;, score=0.581 total time=   0.1s
[CV 3/5] END learning_rate=0.01, n_estimators=10;, score=0.580 total time=   0.1s
[CV 4/5] END learning_rate=0.01, n_estimators=10;, score=0.580 total time=   0.1s
[CV 5/5] END learning_rate=0.01, n_estimators=10;, score=0.580 total time=   0.1s
[CV 1/5] END learning_rate=0.01, n_estimators=50;, score=0.580 total time=   0.5s
[CV 2/5] END learning_rate=0.01, n_estimators=50;, score=0.581 total time=   0.5s
[CV 3/5] END learning_rate=0.01, n_estimators=50;, score=0.580 total time=   0.5s
[CV 4/5] END learning_rate=0.01, n_estimators=50;, score=0.580 total time=   0.5s
[CV 5/5] END learning_rate=0.01, n_estimators=50;, score=0.580 total time=   0.5s
[CV 1/5] END learning_rate=0.01, n_estimators=100;, score=0.580 total time=   0.9s
[CV 2/5] END learning_rate=0.01, n_es

INFO:root:[AdaBoost] 🔍 Best Score: 0.5988
[AdaBoost] 🔍 Best Params: {'learning_rate': 1.0, 'n_estimators': 100}


#### 🔍 AdaBoost Metrics

In [22]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [23]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [24]:
save_model(model, name=file_model_name[model_name])

✅ Model AdaBoost saved to: /home/mlioi/ufc-predictor/models/ab_best.pkl


### 4.6 Naive Bayes Model

#### 🚀 Naive Bayes Training

In [25]:
model_name = 'Naive Bayes'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔═══════════════════════════════════════════╗
║  [Naive Bayes] UFC GridSearchCV Training  ║
╚═══════════════════════════════════════════╝


INFO:root:[Naive Bayes] 🤖 Training...


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ...............var_smoothing=1e-08;, score=0.586 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-08;, score=0.440 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-08;, score=0.589 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-08;, score=0.542 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-08;, score=0.568 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-07;, score=0.588 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-07;, score=0.448 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-07;, score=0.589 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-07;, score=0.542 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-07;, score=0.568 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-06;, score=0.588 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-06;,

INFO:root:[Naive Bayes] 🔍 Best Score: 0.5549
[Naive Bayes] 🔍 Best Params: {'var_smoothing': 1e-05}


#### 🔍 Naive Bayes Metrics

In [26]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [27]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [28]:
save_model(model, name=file_model_name[model_name])

✅ Model Naive Bayes saved to: /home/mlioi/ufc-predictor/models/nb_best.pkl


### 4.7 Gradient Boosting Model

#### 🚀 Gradient Boosting Training

In [29]:
model_name = "Gradient Boosting"
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔═════════════════════════════════════════════════╗
║  [Gradient Boosting] UFC GridSearchCV Training  ║
╚═════════════════════════════════════════════════╝


INFO:root:[Gradient Boosting] 🤖 Training...


Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=10;, score=0.580 total time=   0.2s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=10;, score=0.581 total time=   0.2s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=10;, score=0.580 total time=   0.2s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=10;, score=0.580 total time=   0.2s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=10;, score=0.580 total time=   0.2s
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.578 total time=   1.1s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.585 total time=   1.1s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.588 total time=   1.1s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.582 total time=   1.1s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.581 total t

INFO:root:[Gradient Boosting] 🔍 Best Score: 0.6059
[Gradient Boosting] 🔍 Best Params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}


#### 🔍 Gradient Boosting Metrics

In [30]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [31]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [32]:
save_model(model, name=file_model_name[model_name])

✅ Model Gradient Boosting saved to: /home/mlioi/ufc-predictor/models/gb_best.pkl


### 4.8 Extra Trees Model

#### 🚀 Extra Trees Training

In [33]:
model_name = "Extra Trees"
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔═══════════════════════════════════════════╗
║  [Extra Trees] UFC GridSearchCV Training  ║
╚═══════════════════════════════════════════╝


INFO:root:[Extra Trees] 🤖 Training...


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ...max_depth=None, n_estimators=50;, score=0.607 total time=   0.4s
[CV 2/5] END ...max_depth=None, n_estimators=50;, score=0.585 total time=   0.4s
[CV 3/5] END ...max_depth=None, n_estimators=50;, score=0.602 total time=   0.4s
[CV 4/5] END ...max_depth=None, n_estimators=50;, score=0.578 total time=   0.4s
[CV 5/5] END ...max_depth=None, n_estimators=50;, score=0.572 total time=   0.4s
[CV 1/5] END ..max_depth=None, n_estimators=100;, score=0.610 total time=   0.7s
[CV 2/5] END ..max_depth=None, n_estimators=100;, score=0.587 total time=   0.7s
[CV 3/5] END ..max_depth=None, n_estimators=100;, score=0.593 total time=   0.7s
[CV 4/5] END ..max_depth=None, n_estimators=100;, score=0.583 total time=   0.7s
[CV 5/5] END ..max_depth=None, n_estimators=100;, score=0.581 total time=   0.7s
[CV 1/5] END .....max_depth=10, n_estimators=50;, score=0.595 total time=   0.1s
[CV 2/5] END .....max_depth=10, n_estimators=50;,

INFO:root:[Extra Trees] 🔍 Best Score: 0.5950
[Extra Trees] 🔍 Best Params: {'max_depth': 20, 'n_estimators': 100}


#### 🔍 Extra Trees Metrics

In [34]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [35]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [36]:
save_model(model, name=file_model_name[model_name])

✅ Model Extra Trees saved to: /home/mlioi/ufc-predictor/models/et_best.pkl


### 4.9 Quadratic Discriminant Analysis Model

#### 🚀 Quadratic Discriminant Analysis Training

In [37]:
model_name = "Quadratic Discriminant Analysis"
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔═══════════════════════════════════════════════════════════════╗
║  [Quadratic Discriminant Analysis] UFC GridSearchCV Training  ║
╚═══════════════════════════════════════════════════════════════╝


INFO:root:[Quadratic Discriminant Analysis] 🤖 Training...


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END .....................reg_param=0.1;, score=0.531 total time=   0.0s
[CV 2/5] END .....................reg_param=0.1;, score=0.552 total time=   0.0s
[CV 3/5] END .....................reg_param=0.1;, score=0.578 total time=   0.0s
[CV 4/5] END .....................reg_param=0.1;, score=0.514 total time=   0.0s
[CV 5/5] END .....................reg_param=0.1;, score=0.520 total time=   0.0s
[CV 1/5] END .......................reg_param=1;, score=0.596 total time=   0.0s
[CV 2/5] END .......................reg_param=1;, score=0.578 total time=   0.0s
[CV 3/5] END .......................reg_param=1;, score=0.605 total time=   0.0s
[CV 4/5] END .......................reg_param=1;, score=0.576 total time=   0.0s
[CV 5/5] END .......................reg_param=1;, score=0.576 total time=   0.0s


INFO:root:[Quadratic Discriminant Analysis] 🔍 Best Score: 0.5862
[Quadratic Discriminant Analysis] 🔍 Best Params: {'reg_param': 1}


#### 🔍 Quadratic Discriminant Analysis Metrics

In [38]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [39]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [40]:
save_model(model, name=file_model_name[model_name])

✅ Model Quadratic Discriminant Analysis saved to: /home/mlioi/ufc-predictor/models/qda_best.pkl


### 4.10 Neural Network

#### 🚀 Neural Network Training 

In [41]:
model_name = 'Neural Network'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔══════════════════════════════════════════════╗
║  [Neural Network] UFC GridSearchCV Training  ║
╚══════════════════════════════════════════════╝


INFO:root:[Neural Network] 🤖 Training...


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.15;, score=0.607 total time=   0.8s
[CV 2/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.15;, score=0.581 total time=   1.0s
[CV 3/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.15;, score=0.591 total time=   1.0s
[CV 4/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.15;, score=0.597

INFO:root:[Neural Network] 🔍 Best Score: 0.5986
[Neural Network] 🔍 Best Params: {'activation': 'relu', 'alpha': 0.001, 'batch_size': 32, 'early_stopping': True, 'hidden_layer_sizes': (50, 50), 'learning_rate': 'adaptive', 'learning_rate_init': 0.01, 'momentum': 0.8, 'solver': 'adam', 'validation_fraction': 0.15}


#### 🔍 Neural Network Metrics

In [42]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [43]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [44]:
save_model(model, name=file_model_name[model_name])

✅ Model Neural Network saved to: /home/mlioi/ufc-predictor/models/nn_best.pkl


### 4.11 XGBoost Model

#### 🚀 XGBoost Training 

In [5]:
model_name = 'XGBoost'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔═══════════════════════════════════════╗
║  [XGBoost] UFC GridSearchCV Training  ║
╚═══════════════════════════════════════╝


INFO:root:[XGBoost] 🤖 Training...


Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.580 total time=   0.5s
[CV 2/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.580 total time=   0.1s
[CV 3/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.583 total time=   0.1s
[CV 4/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.580 total time=   0.1s
[CV 5/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.580 total time=   0.1s
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0;, score=0.580 total time=   0.1s
[CV 2/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0;, score=0.580 total time=   0.1s
[CV 3/5] END c

INFO:root:[XGBoost] 🔍 Best Score: 0.6074
[XGBoost] 🔍 Best Params: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}


#### 🔍 XGBoost Metrics

In [9]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [10]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [11]:
save_model(model, name=file_model_name[model_name])

✅ Model XGBoost saved to: /home/mlioi/ufc-predictor/models/xgb_best.pkl


<div style="text-align: center;">
     <img src="../img/ufc_logo.png" width="800" /> 
</div>